# Custom unitary operation

Using a unitary operation is sometimes preferred to gates when designing quantum algorithms that are more abstract, have oracles, or have no exactly known gate set. 

You can now execute custom unitary operations within CUDA-Q kernels. The following code example shows how to specify a custom unitary operation as a NumPy array, name it, and then use it in a kernel. A custom standard X gate is specified as a 2×2 unitary matrix with rows [0,1] and [1,0]. The example also demonstrates how custom unitaries can be applied using a controlled operation of one or more qubits.

In [4]:
import numpy as np
import cudaq
 
cudaq.register_operation("custom_x", np.array([0, 1, 1, 0]))
 
 
@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    custom_x(qubits[0])
    custom_x.ctrl(qubits[0], qubits[1])
 
 
counts = cudaq.sample(kernel)

print(counts)

{ 11:505 00:495 }



## Multi-qubit operations.

In [2]:
import cudaq
import numpy as np

# Create and test a custom CNOT operation.
cudaq.register_operation("my_cnot", np.array([1, 0, 0, 0,
                                              0, 1, 0, 0,
                                              0, 0, 0, 1,
                                              0, 0, 1, 0]))

@cudaq.kernel
def bell_pair():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    my_cnot(qubits[0], qubits[1]) # `my_cnot(control, target)`


count= cudaq.sample(bell_pair)

print(count)# prints { 11:500 00:500 } (exact numbers will be random)

# Construct a custom unitary matrix for X on the first qubit and Y
# on the second qubit.
X = np.array([[0,  1 ], [1 , 0]])
Y = np.array([[0, -1j], [1j, 0]])
XY = np.kron(X, Y)

# Register the custom operation
cudaq.register_operation("my_XY", XY)

@cudaq.kernel
def custom_xy_test():
    qubits = cudaq.qvector(2)
    my_XY(qubits[0], qubits[1])
    y(qubits[1]) # undo the prior Y gate on qubit 1


count_2=cudaq.sample(custom_xy_test)

print(count_2) # prints { 10:1000 }

{ 11:510 00:490 }

{ 10:1000 }

